In [12]:

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
# import xgboost as xgb
# import lightgbm as lgb
# from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef

df = pd.read_csv('../data/raw/data-923.csv')
# df = df.drop(columns=['num'])

label_encoder = LabelEncoder()
df['A_encoded'] = label_encoder.fit_transform(df['cate'])
y = df['A_encoded']
X = df[['en1','en2','en3','en4', 'en5', 'en6', 'en7', 'en8', 'en9', 'en10', 'en11', 'en12', 'en13', 'en14', 'en15']]

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42,stratify=y)

# 定义分类器及其参数
classifiers = {
    'DecisionTree': DecisionTreeClassifier(criterion='entropy',random_state=42),
    # SVM 支持多种核函数，包括线性核（linear）、多项式核（poly）、径向基函数（RBF，rbf）和sigmoid核。对于非线性问题，通常使用非线性核函数，RBF 是最常用的选择。
    # 'SVM': SVC(probability=True, kernel='rbf',random_state=42),
    # 'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    # 'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42),
    'KNeighbors': KNeighborsClassifier(weights='distance'),
    'GaussianNB': GaussianNB(var_smoothing=1e-2),
    # 'AdaBoost': AdaBoostClassifier(learning_rate=0.1,random_state=42),
    'MLP (Neural Network)': MLPClassifier(hidden_layer_sizes=(100,), max_iter=100,learning_rate_init=0.01, random_state=42),
    # 'Logistic Regression': LogisticRegression(),
    'LDA': LinearDiscriminantAnalysis(),
    # 'XGBoost': xgb.XGBClassifier(eval_metric='mlogloss'),
    # 'LightGBM': lgb.LGBMClassifier(),
    # 'CatBoost': CatBoostClassifier(silent=True),
    'Extra Trees': ExtraTreesClassifier()
}

# 封装训练和评估模型的函数
def train_evaluate(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    kappa = matthews_corrcoef(y_test, y_pred)

    print(f"模型：{clf.__class__.__name__}")
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'F1 Score: {f1}')
    print(f'Kappa Coefficient: {kappa}')
    print('------')
    
    # print(', '.join(y_test.astype(str)))
    # print(y_pred)

# 创建一个投票分类器
voting_clf = VotingClassifier(estimators=[(name, clf) for name, clf in classifiers.items()], voting='soft')

for name, clf in classifiers.items():
    train_evaluate(clf,X_train,y_train,X_test,y_test)

train_evaluate(voting_clf,X_train,y_train,X_test,y_test)


模型：DecisionTreeClassifier
Accuracy: 0.9166666666666666
Recall: 0.9230769230769231
Precision: 0.9358974358974359
F1 Score: 0.9076923076923076
Kappa Coefficient: 0.9130451096264045
------
模型：KNeighborsClassifier
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
Kappa Coefficient: 1.0
------
模型：GaussianNB
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
Kappa Coefficient: 1.0
------
模型：MLPClassifier
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
Kappa Coefficient: 1.0
------
模型：LinearDiscriminantAnalysis
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
Kappa Coefficient: 1.0
------
模型：ExtraTreesClassifier
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
Kappa Coefficient: 1.0
------
模型：VotingClassifier
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
Kappa Coefficient: 1.0
------


# Oil Type Discrimination - Analysis & Visualization

## Purpose
This notebook contains exploratory data analysis, algorithm comparison, and publication-ready visualizations used in the paper.

## Paper Figures Generated
| Figure | Description | Cell |
|--------|-------------|------|
| Fig 1 | Nature-style 2D scatter plot | [Nature-style Plot] |
| Fig 2 | LDA Scree plot | [LDA Scree Plot] |
| Fig 3 | Feature correlation heatmap | [Correlation Analysis] |
| Fig 4 | ROC curves by LDA dimensions | [ROC Analysis] |
| Fig 5 | 3D interactive visualization | [3D Visualization] |

## Algorithms Tested
- Decision Tree, KNN, GaussianNB, MLP
- LDA, CatBoost, Extra Trees
- SVM, Random Forest, Gradient Boosting

## Input Data
- File: `../data/data-923.csv`
- Samples: 78 (13 oil types × 6 replicates)
- Features: 15 enzyme absorbance values (en1-en15)

## Output
All figures are saved to the current directory in PDF/SVG format.

## Note
The core plotting functions from this notebook have been extracted to `src/visualization/` for reusability.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef
import matplotlib.pyplot as plt
import os

# 创建输出目录
os.makedirs("output", exist_ok=True)

# ============================================================
# Nature 风格可视化函数（右侧图例）
# ============================================================
def plot_nature_scatter(X2d, y_labels, title="Nature-style Plot", save_name=None):
    plt.rcParams.update({
        "font.family": "Arial",
        "figure.dpi": 300,
        "savefig.dpi": 300,
        "axes.linewidth": 1.2,
        "axes.labelsize": 14,
        "axes.titlesize": 15,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        "xtick.major.width": 1.0,
        "ytick.major.width": 1.0,
        "legend.frameon": False,
        "axes.facecolor": "white",
        "axes.edgecolor": "black",
    })

    nature_colors = [
        "#4C72B0", "#DD8452", "#55A868",
        "#C44E52", "#8172B3", "#937860",
        "#DA8BC3", "#8C8C8C", "#64B5CD"
    ]

    plt.figure(figsize=(6.5, 5))
    unique_classes = np.unique(y_labels)

    for i, cls in enumerate(unique_classes):
        idx = (y_labels == cls)
        plt.scatter(
            X2d[idx, 0], X2d[idx, 1],
            s=55, alpha=0.78,
            color=nature_colors[i % len(nature_colors)],
            edgecolor="black", linewidth=0.35,
            label=str(cls)
        )

    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    plt.xlabel("Component 1", labelpad=6)
    plt.ylabel("Component 2", labelpad=6)
    plt.title(title, pad=10)

    # 图例在右侧
    plt.legend(
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        frameon=False,
        borderaxespad=0,
        handletextpad=0.3
    )

    plt.tight_layout()
    if save_name:
        plt.savefig(f"output/{save_name}.pdf", bbox_inches="tight")
        plt.savefig(f"output/{save_name}.svg", bbox_inches="tight")
    plt.show()


# ============================================================
# 读取数据
# ============================================================
df = pd.read_csv("../data/raw/data-923.csv")
label_encoder = LabelEncoder()
df["cate_str"] = df["cate"].astype(str)
df["cate_encoded"] = label_encoder.fit_transform(df["cate_str"])

y = df["cate_encoded"]
X = df[[f"en{i}" for i in range(1, 16)]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ============================================================
# PCA / LDA 降维
# ============================================================
use_lda = True  # True 使用 LDA，False 使用 PCA

if use_lda:
    # LDA 降维，最多 n_classes-1
    lda = LinearDiscriminantAnalysis(n_components= min(8, len(np.unique(y))-1))
    X_reduced = lda.fit_transform(X_scaled, y)
else:
    # PCA 降维
    pca = PCA(n_components=8)
    X_reduced = pca.fit_transform(X_scaled)

# ============================================================
# 划分训练集（全部降维后的维度用于训练）
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.3, stratify=y, random_state=42
)

# ============================================================
# 模型列表
# ============================================================
models = {
    "Decision Tree": DecisionTreeClassifier(criterion="entropy"),
    #"KNN": KNeighborsClassifier(weights="distance"),
    "Naive Bayes": GaussianNB(var_smoothing=1e-2),
    #"MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=150, learning_rate_init=0.01),
    #"CatBoost": CatBoostClassifier(silent=True),
    #"Extra Trees": ExtraTreesClassifier()
}

# ============================================================
# 训练 + 输出指标 + 可视化（前两维）
# ============================================================
for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_str = label_encoder.inverse_transform(y_pred)

    # 输出指标
    print(f"\n模型：{name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}; "
          f"Recall: {recall_score(y_test, y_pred, average='macro')}; "
          f"Precision: {precision_score(y_test, y_pred, average='macro')}; "
          f"F1: {f1_score(y_test, y_pred, average='macro')}; "
          f"MCC(Kappa): {matthews_corrcoef(y_test, y_pred)}")

    # 可视化：只用降维后的前两维定义 XY
    plot_nature_scatter(
        X_reduced[:, :2],
        label_encoder.inverse_transform(clf.predict(X_reduced)),
        title=f"{name} Prediction (2D Visualization)",
        save_name=f"{name}_2D_Nature"
    )


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import chi2
import os

# ===================== 工具函数：生成置信椭球 (保持不变) =====================
def get_confidence_ellipsoid(x, y, z, confidence=0.9):
    data = np.vstack((x, y, z))
    if data.shape[1] < 4: return None, None, None
    mean = np.mean(data, axis=1)
    cov = np.cov(data)
    eigvals, eigvecs = np.linalg.eigh(cov)
    scale_factor = np.sqrt(chi2.ppf(confidence, df=3))
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, np.pi, 20)
    x_sphere = np.outer(np.cos(u), np.sin(v))
    y_sphere = np.outer(np.sin(u), np.sin(v))
    z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
    sphere_points = np.stack((x_sphere.flatten(), y_sphere.flatten(), z_sphere.flatten()))
    radii = np.sqrt(np.maximum(eigvals, 0)) * scale_factor
    transformed = (eigvecs @ np.diag(radii) @ sphere_points).T + mean
    return transformed[:, 0], transformed[:, 1], transformed[:, 2]

# ===================== 1. 数据准备 =====================
df = pd.read_csv("../data/raw/data-923.csv")
label_encoder = LabelEncoder()
df["cate_encoded"] = label_encoder.fit_transform(df["cate"].astype(str))
y = df["cate_encoded"]
X = df[[f"en{i}" for i in range(1, 16)]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# LDA 降维
lda = LinearDiscriminantAnalysis(n_components=min(3, len(np.unique(y)) - 1))
X_reduced_all = lda.fit_transform(X_scaled, y)
X_plot_3d = X_reduced_all[:, :3]

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced_all, y, test_size=0.3, stratify=y, random_state=42
)

# ===================== 2. 模型训练与指标计算 (测试集) =====================
model = GaussianNB(var_smoothing=1e-2)
model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_test)

metric_map = {}
unique_labels_encoded = np.unique(y_test)
unique_labels_str = label_encoder.inverse_transform(unique_labels_encoded)

for i, label_str in enumerate(unique_labels_str):
    tp = cm[i, i]
    total_true = cm[i, :].sum()
    recall = tp / total_true if total_true > 0 else 0
    total_pred = cm[:, i].sum()
    precision = tp / total_pred if total_pred > 0 else 0
    metric_map[label_str] = f" | R:{recall:.2f}, P:{precision:.2f}"
    # print(f"Class {label_str}: Recall={recall:.2f}, Precision={precision:.2f}") # 可以取消注释查看

# --- 准备全量数据的绘图数据 ---
all_probs = model.predict_proba(X_reduced_all)
confidence_scores = np.max(all_probs, axis=1)
y_pred = model.predict(X_reduced_all)
y_pred_str = label_encoder.inverse_transform(y_pred)
y_true_str = label_encoder.inverse_transform(y)

df_plot = pd.DataFrame({
    "Comp1": X_plot_3d[:, 0], "Comp2": X_plot_3d[:, 1], "Comp3": X_plot_3d[:, 2],
    "True Label": y_true_str, "Predicted Label": y_pred_str, "Confidence": confidence_scores,
    "Is_Correct": y == y_pred
})
df_plot["Status"] = np.where(df_plot["Is_Correct"], "Correct", "Misclassified")

# ===================== 3. 绘图：颜色和图例优化 =====================
output_dir = "output/3D_Optimized_Legend"
os.makedirs(output_dir, exist_ok=True)

# --- 关键修改：生成足够多的不重复颜色 ---
# 如果类别很多，可能需要 Plotly 之外的调色板，例如 matplotlib 的'tab20'或'Paired'
# 这里使用 Plotly 默认的定性色板，它通常能处理10个左右的不同颜色
# 如果类别数大于len(px.colors.qualitative.Plotly)，颜色会重复。
# 更好的做法是生成一个更长的颜色列表，例如：
num_classes = len(unique_labels_str)
colors_base = px.colors.qualitative.Alphabet if num_classes > 10 else px.colors.qualitative.Plotly
colors_extended = colors_base * ((num_classes // len(colors_base)) + 1)
color_map = {label: colors_extended[i] for i, label in enumerate(unique_labels_str)}


fig = go.Figure()
print("正在生成图表...")

# 添加一个额外的空散点图，专门用于图例中的 "Misclassified" 标记
# 这个标记将是统一的，不区分具体类别，仅代表"预测错误"这一概念
fig.add_trace(go.Scatter3d(
    x=[None], y=[None], z=[None], # 不显示实际点
    mode='markers',
    marker=dict(symbol='diamond-open', color='rgba(0,0,0,0.5)', size=8), # 统一的灰色菱形
    name='Misclassified Points', # 图例名称
    showlegend=True,
    legendgroup='_misc', # 单独分组
    hoverinfo='skip'
))


for label in unique_labels_str:
    subset = df_plot[df_plot["True Label"] == label]
    color = color_map[label]
    metrics_suffix = metric_map.get(label, "")
    
    # --- A. 绘制 90% 置信椭球 (主图例项，带有颜色色块和指标) ---
    ex, ey, ez = get_confidence_ellipsoid(
        subset["Comp1"].values, subset["Comp2"].values, subset["Comp3"].values, confidence=0.9
    )
    if ex is not None:
        fig.add_trace(go.Mesh3d(
            x=ex, y=ey, z=ez,
            alphahull=0, opacity=0.15, color=color,
            name=f"{label}{metrics_suffix}", # 显示类别和指标
            legendgroup=label,
            showlegend=False,
            hoverinfo='skip'
        ))
        # 更好的方法是添加一个不可见的散点图来控制图例标记
        fig.add_trace(go.Scatter3d(
            x=[None], y=[None], z=[None], # 不显示点
            mode='markers',
            marker=dict(color=color, symbol='circle', size=10), # 圆形色块
            name=f"{label}{metrics_suffix}", # 再次命名，这个trace是用来显示图例的
            legendgroup=label,
            showlegend=True, # 显示图例
            hoverinfo='skip',
            visible='legendonly', # 默认不显示在图上，只显示在图例中
            legendrank=0 # 确保它排在最前面
        ))


    # --- B. 绘制散点 (区分正确与错误) ---
    # 正确分类的散点，不显示图例，只显示在图上
    sub_subset_correct = subset[subset["Status"] == "Correct"]
    if not sub_subset_correct.empty:
        custom_data_correct = np.stack((
            sub_subset_correct["True Label"], sub_subset_correct["Predicted Label"],
            sub_subset_correct["Status"], sub_subset_correct["Confidence"]
        ), axis=-1)
        fig.add_trace(go.Scatter3d(
            x=sub_subset_correct["Comp1"], y=sub_subset_correct["Comp2"], z=sub_subset_correct["Comp3"],
            mode='markers',
            name=f"{label} (Correct)", # 这个name会显示在hover中
            legendgroup=label,
            showlegend=False, # 不在图例中显示
            marker=dict(
                size=sub_subset_correct["Confidence"] * 10 + 2,
                color=color,
                symbol='circle',
                opacity=0.9,
                line=dict(width=0)
            ),
            customdata=custom_data_correct,
            hovertemplate=(
                "<b>True Label: %{customdata[0]}</b><br>" +
                "Predicted: %{customdata[1]}<br>" +
                "Status: %{customdata[2]}<br>" +
                "Confidence: %{customdata[3]:.1%}<br>" +
                "<extra></extra>"
            )
        ))
    
    # 错误分类的散点，现在是统一的灰色菱形，会有一个单独的图例项
    sub_subset_misclassified = subset[subset["Status"] == "Misclassified"]
    if not sub_subset_misclassified.empty:
        custom_data_misclassified = np.stack((
            sub_subset_misclassified["True Label"], sub_subset_misclassified["Predicted Label"],
            sub_subset_misclassified["Status"], sub_subset_misclassified["Confidence"]
        ), axis=-1)
        fig.add_trace(go.Scatter3d(
            x=sub_subset_misclassified["Comp1"], y=sub_subset_misclassified["Comp2"], z=sub_subset_misclassified["Comp3"],
            mode='markers',
            name=f"{label} (Misclassified)", # 仍然用于hover
            legendgroup='_misc', # 分配到统一的错误分类组
            showlegend=False, # 不显示在图例中，因为我们已经有统一的 "Misclassified Points"
            marker=dict(
                size=sub_subset_misclassified["Confidence"] * 10 + 2,
                color=color, # 颜色仍然是其True Label的颜色
                symbol='diamond-open', # 空心菱形
                opacity=0.9,
                line=dict(width=0)
            ),
            customdata=custom_data_misclassified,
            hovertemplate=(
                "<b>True Label: %{customdata[0]}</b><br>" +
                "Predicted: %{customdata[1]}<br>" +
                "Status: %{customdata[2]}<br>" +
                "Confidence: %{customdata[3]:.1%}<br>" +
                "<extra></extra>"
            )
        ))

# ===================== 4. 布局优化 =====================
fig.update_layout(
    title="3D Analysis: Per-Class Test Set Metrics in Legend (Optimized)",
    scene=dict(
        xaxis_title='LDA Component 1',
        yaxis_title='LDA Component 2',
        zaxis_title='LDA Component 3',
        aspectmode='cube'
    ),
    width=1200,
    height=900,
    legend=dict(
        title="Class (R=Recall, P=Precision)",
        itemsizing='constant',
        groupclick="toggleitem"
    )
)

html_file = os.path.join(output_dir, "Optimized_Legend_Analysis.html")
fig.write_html(html_file)
print(f"Saved: {html_file}")


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc, confusion_matrix

# ===================== 自定义颜色 =====================
custom_colors = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
    "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf",
    "#aec7e8", "#ffbb78", "#98df8a"
]

# ===================== 数据读取与预处理 =====================
df = pd.read_csv("../data/raw/data-923.csv")
cols = [f"en{i}" for i in range(1,16)]
X = df[cols].values
y_raw = df["cate"].astype(str).values
le = LabelEncoder()
y = le.fit_transform(y_raw)
class_names = le.classes_
n_classes = len(class_names)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
all_y_true, all_y_pred, all_y_prob = [], [], []

for train_idx, test_idx in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    lda = LinearDiscriminantAnalysis(n_components=n_classes-1)
    X_train_lda = lda.fit_transform(X_train, y_train)
    X_test_lda = lda.transform(X_test)

    model = GaussianNB()
    model.fit(X_train_lda, y_train)
    y_pred = model.predict(X_test_lda)
    y_prob = model.predict_proba(X_test_lda)

    all_y_true.append(y_test)
    all_y_pred.append(y_pred)
    all_y_prob.append(y_prob)

y_true_all = np.concatenate(all_y_true)
y_pred_all = np.concatenate(all_y_pred)
y_prob_all = np.concatenate(all_y_prob)




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 确保输出目录存在
os.makedirs("output", exist_ok=True)

# 特征数据
cols = [f"en{i}" for i in range(1,16)]
X_df = df[cols]

# 相关系数矩阵
corr_matrix = X_df.corr()

# 创建一个 mask，只显示上三角的颜色
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

fig, ax = plt.subplots(figsize=(10,8))

# 上三角热力图（颜色）
sns.heatmap(
    corr_matrix,
    mask=~mask,            # 上三角
    annot=False,           # 不显示数字
    cmap="coolwarm",
    vmin=-1, vmax=1,
    cbar=True,
    linewidths=0.3,
    linecolor='gray',
    square=True,
    ax=ax
)

# 下三角数字显示
for i in range(corr_matrix.shape[0]):
    for j in range(i):
        ax.text(j+0.5, i+0.5,
                f"{corr_matrix.iloc[i,j]:.2f}",
                ha='center', va='center', fontsize=9, color='black')

# 坐标轴和标题
ax.set_xticks(np.arange(corr_matrix.shape[0])+0.5)
ax.set_yticks(np.arange(corr_matrix.shape[0])+0.5)
ax.set_xticklabels(corr_matrix.columns, rotation=45, ha='right', fontsize=10)
ax.set_yticklabels(corr_matrix.columns, rotation=0, fontsize=10)
ax.set_title("Feature Correlation Matrix", fontsize=13)

# 极细边框
for spine in ax.spines.values():
    spine.set_linewidth(0.3)

ax.grid(
         linestyle='-', 
         alpha=0.7, 
         color='lightgray', 
         linewidth=0.3)

plt.tight_layout()
plt.savefig("output/Feature_Correlation_Matrix_Professional.pdf", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# 确保输出目录存在
os.makedirs("output", exist_ok=True)

# 假设 explained_var 和 cum_var 已经由 LDA fit 得到
explained_var = lda.explained_variance_ratio_
cum_var = np.cumsum(explained_var)
ld_indices = np.arange(1, len(explained_var)+1)

fig, ax1 = plt.subplots(figsize=(6,4))

# ------------------- 左轴：柱状图 -------------------
bars = ax1.bar(ld_indices, explained_var, color="#1f77b4", alpha=0.85,
               edgecolor='k', linewidth=0, label="Individual LD")
ax1.set_xlabel("LDA Component", fontsize=10)
ax1.set_ylabel("Explained Variance Ratio", fontsize=10, color="#1f77b4")
ax1.tick_params(axis='y', labelcolor="#1f77b4", labelsize=9, width=0.5, length=4)
ax1.tick_params(axis='x', width=0.5, length=4)
ax1.set_xticks(ld_indices)
ax1.set_ylim(0, max(explained_var)*1.3)

# 柱顶标注每个 LD 的贡献率
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., 1.02*height,
             f'{height:.2f}', ha='center', va='bottom', fontsize=7)

# ------------------- 右轴：累计折线 -------------------
ax2 = ax1.twinx()
ax2.plot(ld_indices, cum_var*100, color="#ff7f0e", marker='o', markersize=5,
         linewidth=1.8, label="Cumulative Explained")
ax2.set_ylabel("Cumulative Explained Variance (%)", fontsize=10, color="#ff7f0e")
ax2.tick_params(axis='y', labelcolor="#ff7f0e", labelsize=9, width=0.5, length=4)
ax2.set_ylim(0, 105)

# 标注达到80%累计解释率的LD
threshold = 80
first_above = np.where(cum_var*100 >= threshold)[0][0] + 1
ax2.axvline(x=first_above, color='grey', linestyle='--', linewidth=0.5)
ax2.text(first_above+0.3, 30, f'{threshold}% variance at LD{first_above}',
         color='grey', fontsize=8)

# ------------------- 坐标轴极细 -------------------
for spine in ax1.spines.values():
    spine.set_linewidth(0.2)
for spine in ax2.spines.values():
    spine.set_linewidth(0.2)

ax1.grid(
         linestyle='-', 
         alpha=0.7, 
         color='lightgray', 
         linewidth=0.3)
ax2.grid(
         linestyle='-', 
         alpha=0.7, 
         color='lightgray', 
         linewidth=0.3)
# ------------------- 网格和美化 -------------------
fig.suptitle("LDA Scree Plot", fontsize=12)
fig.tight_layout()

# ------------------- 高精度输出 -------------------
plt.savefig("output/Fig2a_LDA_scree_Nature_Final.pdf", dpi=300)
plt.show()


In [ ]:
import pandas as pd
import os

# 确保输出目录存在
os.makedirs("output", exist_ok=True)

# ------------------- 保存 LDA Scree 数据到 CSV -------------------
df_lda_scree = pd.DataFrame({
    "LD": ld_indices,
    "explained_variance_ratio": explained_var,
    "cumulative_explained_variance": cum_var,
    "cumulative_explained_variance_%": cum_var * 100
})

df_lda_scree.to_csv("output/Fig2a_LDA_scree_data.csv", index=False)
print("Saved: output/Fig2a_LDA_scree_data.csv")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc
import os

# 确保输出目录存在
os.makedirs("output", exist_ok=True)

# ===================== 数据读取与预处理 =====================
df = pd.read_csv("../data/raw/data-923.csv")
cols = [f"en{i}" for i in range(1,16)]
X = df[cols].values
y_raw = df["cate"].astype(str).values
le = LabelEncoder()
y = le.fit_transform(y_raw)
class_names = le.classes_
n_classes = len(class_names) # 类别数

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===================== 遍历维度与性能评估 =====================

# LDA 最大维度为 n_classes - 1 (这里是 13 - 1 = 12)
max_lda_dim = n_classes - 1 

# 存储每种维度下的平均 ROC 数据
results_by_dim = {}

# 使用 Plotly 的颜色，确保不同维度有区分
colors_extended = px.colors.qualitative.Plotly * 2 

# 遍历所有可能的 LDA 维度
for n_dim in range(1, max_lda_dim + 1):
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_true_all_cv, y_prob_all_cv = [], []

    # 进行 5 折交叉验证
    for train_idx, test_idx in kf.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # 1. LDA 降维
        lda = LinearDiscriminantAnalysis(n_components=n_dim)
        X_train_lda = lda.fit_transform(X_train, y_train)
        X_test_lda = lda.transform(X_test)

        # 2. GaussianNB 模型训练与预测
        model = GaussianNB()
        model.fit(X_train_lda, y_train)
        
        # 收集真实标签和预测概率
        y_prob = model.predict_proba(X_test_lda)
        y_true_all_cv.append(y_test)
        y_prob_all_cv.append(y_prob)

    # 3. 汇总 5 折 CV 结果
    y_true_all = np.concatenate(y_true_all_cv)
    y_prob_all = np.concatenate(y_prob_all_cv)

    # 4. 计算整体的 AUC 和 ROC 曲线
    y_true_bin = label_binarize(y_true_all, classes=np.arange(n_classes))
    
    # 将多分类问题视为 One-vs-All，并压平成一维进行整体 ROC 计算
    fpr, tpr, _ = roc_curve(y_true_bin.ravel(), y_prob_all.ravel())
    roc_auc = auc(fpr, tpr)

    # 存储结果
    results_by_dim[n_dim] = {
        'fpr': fpr,
        'tpr': tpr,
        'auc': roc_auc,
        'color': colors_extended[n_dim - 1] # 确保每条曲线颜色不同
    }

# ===================== 绘图：对比所有维度的 ROC 曲线 =====================
fig, ax = plt.subplots(figsize=(8,6))
ax.plot([0,1], [0,1], '--', color='grey', lw=0.8)

# 绘制每种维度的 ROC 曲线
best_auc = -1
best_dim = 0

for n_dim, result in results_by_dim.items():
    label = f"LDA Dim: {n_dim} (AUC={result['auc']:.3f})"
    ax.plot(result['fpr'], result['tpr'], 
            color=result['color'], 
            lw=1.5, 
            label=label,
            alpha=0.7)
    
    # 记录最佳性能的维度
    if result['auc'] > best_auc:
        best_auc = result['auc']
        best_dim = n_dim

# 坐标轴、标题、网格
ax.set_xlabel("False Positive Rate", fontsize=12)
ax.set_ylabel("True Positive Rate", fontsize=12)
ax.set_title(f"ROC Curve Comparison (5-Fold CV) - Best Dim: {best_dim} (AUC={best_auc:.3f})", fontsize=14)
ax.tick_params(axis='both', labelsize=11, width=0.5, length=4)

# 极细坐标轴
for spine in ax.spines.values():
    spine.set_linewidth(0.4)

# 调整图例位置，避免遮挡曲线
ax.legend(fontsize=9, loc="lower right", frameon=True, framealpha=0.95, ncols=2)

ax.grid(linestyle='--', alpha=0.5, color='lightgray', linewidth=0.3)

plt.tight_layout()
plt.savefig("output/Fig2b_ROC_LDA_Dimension_Comparison.pdf", dpi=300)
plt.show()

print(f"\n最佳 LDA 维度为: {best_dim}，对应的平均 AUC 值为: {best_auc:.3f}")


In [ ]:
import pandas as pd
import os

# 确保输出目录存在
os.makedirs("output", exist_ok=True)

# ===================== 保存 AUC 汇总结果 =====================
auc_summary = pd.DataFrame([
    {"lda_dim": n_dim, "auc": result["auc"]}
    for n_dim, result in results_by_dim.items()
])

auc_summary.to_csv(
    "output/Fig2b_LDA_Dimension_AUC_Summary.csv",
    index=False
)
print("Saved: output/Fig2b_LDA_Dimension_AUC_Summary.csv")

# ===================== 保存 ROC 曲线数据 =====================
roc_records = []

for n_dim, result in results_by_dim.items():
    fpr = result["fpr"]
    tpr = result["tpr"]
    
    for i in range(len(fpr)):
        roc_records.append({
            "lda_dim": n_dim,
            "fpr": fpr[i],
            "tpr": tpr[i]
        })

df_roc = pd.DataFrame(roc_records)

df_roc.to_csv(
    "output/Fig2b_LDA_Dimension_ROC_Curves.csv",
    index=False
)
print("Saved: output/Fig2b_LDA_Dimension_ROC_Curves.csv")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import os

# 确保输出目录存在
os.makedirs("output", exist_ok=True)

# ===================== 获取最后一折测试集结果 =====================
# X_test, y_test, y_pred 是循环中最后一折的值
# 如果你希望展示特定折，可改成对应索引
cm = confusion_matrix(y_test, y_pred)
n_classes = cm.shape[0]

# ===================== 绘图 =====================
fig, ax = plt.subplots(figsize=(8,6))
ax.set_facecolor("white")
diag_color = "#ff7f0e"  # 对角线颜色

# 绘制对角线矩形
for i in range(n_classes):
    ax.add_patch(plt.Rectangle((i, i), 1, 1, fill=True, color=diag_color, alpha=0.8))

# 添加绝对值注释
for i in range(n_classes):
    for j in range(n_classes):
        ax.text(j+0.5, i+0.5, str(cm[i,j]),
                ha='center', va='center', fontsize=10, color='black')

# 设置刻度和标签
ax.set_xticks(np.arange(n_classes)+0.5)
ax.set_yticks(np.arange(n_classes)+0.5)
ax.set_xticklabels(class_names, ha='right', fontsize=10)
ax.set_yticklabels(class_names, fontsize=10)

# 坐标轴和标题
ax.set_xlabel("Predicted", fontsize=11)
ax.set_ylabel("True", fontsize=11)
ax.set_title("Confusion Matrix — Last Fold Test Set", fontsize=13)

# 隐藏边框
for spine in ax.spines.values():
    spine.set_visible(False)

ax.set_xlim(0, n_classes)
ax.set_ylim(0, n_classes)
ax.invert_yaxis()
ax.set_aspect('equal')

ax.grid(
         linestyle='--', 
         alpha=0.7, 
         color='lightgray', 
         linewidth=0.3)

plt.tight_layout()
plt.savefig("output/ConfusionMatrix_TestSet.pdf", dpi=300)
plt.show()
